In [26]:
!pip install catboost

In [27]:
!pip install eli5

In [28]:
import pandas as pd
import numpy as np
import math
import json
import re
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import catboost as ctb
import eli5
from sklearn.model_selection import GridSearchCV

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
#import dataset prepared in the previous step
df = pd.read_csv("/content/drive/MyDrive/Udacity/df_final.csv")
df =df.fillna(0)
df.head()

,Unnamed: 0,person,event,time,offer_id,cumsum_offer_received,cumsum_offer_viewed,cumsum_offer_completed,cumsum_transaction_wo_offer,cumsum_amount_wo_offer,...,duration_hours,channels_email,channels_mobile,channels_web,channels_social,offer_type_bogo,offer_type_discount,last_amount,last_reward,completed
0,0,0009655768c64bdeb2e877511632db8f,offer received,408,f19421c1d4aa40978ebb69ca19b0e20d,2.0,2.0,0.0,1.0,22.16,...,120.0,1.0,1.0,1.0,1.0,1.0,0.0,0.00,0.0,True
1,1,0009655768c64bdeb2e877511632db8f,offer received,504,fafdcd668e3743c1bb461111dcafc2a4,3.0,3.0,1.0,1.0,22.16,...,240.0,1.0,1.0,1.0,1.0,0.0,1.0,8.57,5.0,True
2,3,00116118485d4dfda04fdbaba9a87b5c,offer received,168,f19421c1d4aa40978ebb69ca19b0e20d,0.0,0.0,0.0,0.0,0.00,...,120.0,1.0,1.0,1.0,1.0,1.0,0.0,10.27,2.0,False
3,4,0011e0d4e6b944f998e987f904e8c1e5,offer received,168,2298d6c36e964ae4a3e7e9706d1fb8c2,1.0,1.0,0.0,1.0,13.49,...,168.0,1.0,1.0,1.0,1.0,0.0,1.0,0.00,0.0,True
4,5,0011e0d4e6b944f998e987f904e8c1e5,offer received,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,3.0,3.0,1.0,1.0,13.49,...,240.0,1.0,0.0,1.0,0.0,0.0,1.0,11.93,3.0,False


In [31]:
#rename columns
df.rename(columns={'age_65<': 'age_higher_than_65'}, inplace=True)
df.rename(columns={'age_<26': 'age_lower_than_26'}, inplace=True)

**Test/train split** <br>
Split dataset to train (80%) and test (20%) <br>
Sort by time, so as to take only last transaction for testing for every user.

In [32]:
df = df.sort_values(by='time')
df.head()

,Unnamed: 0,person,event,time,offer_id,cumsum_offer_received,cumsum_offer_viewed,cumsum_offer_completed,cumsum_transaction_wo_offer,cumsum_amount_wo_offer,...,duration_hours,channels_email,channels_mobile,channels_web,channels_social,offer_type_bogo,offer_type_discount,last_amount,last_reward,completed
9550,11199,3d4191b097e74f099a9be33af7d67d6d,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0.0,0.0,0.0,0.0,...,240.0,1.0,1.0,1.0,1.0,0.0,1.0,5.46,5.0,True
10213,11976,417c8e42cba54dd0ba1ea7ee079ef87a,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0.0,0.0,0.0,0.0,...,240.0,1.0,1.0,1.0,1.0,0.0,1.0,6.02,4.0,True
10216,11979,4180446d6f81480cbc467b969b9559bc,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.0,0.0,0.0,0.0,0.0,...,168.0,1.0,1.0,1.0,0.0,1.0,0.0,12.02,10.0,True
28775,33606,b6859fb6765f401882193edfe8b76360,offer received,0,fafdcd668e3743c1bb461111dcafc2a4,0.0,0.0,0.0,0.0,0.0,...,240.0,1.0,1.0,1.0,1.0,0.0,1.0,0.00,0.0,True
10219,11982,4181890d96874b49a39fbcb22cb48853,offer received,0,2906b810c7d4411798c6938adc9daaa5,0.0,0.0,0.0,0.0,0.0,...,168.0,1.0,1.0,1.0,0.0,0.0,1.0,20.77,20.0,True


In [33]:
# Calculate the index to split the data
split_index = int(0.8 * len(df))

# Split the data into training and testing sets
train_data = df.iloc[:split_index]
test_data = df.iloc[split_index:]

In [34]:
test_data.head()

,Unnamed: 0,person,event,time,offer_id,cumsum_offer_received,cumsum_offer_viewed,cumsum_offer_completed,cumsum_transaction_wo_offer,cumsum_amount_wo_offer,...,duration_hours,channels_email,channels_mobile,channels_web,channels_social,offer_type_bogo,offer_type_discount,last_amount,last_reward,completed
11133,13029,4715bcde100647d6bbcc9b4f1e39df92,offer received,504,ae264e3637204a6fb9bb56bc8210ddfd,2.0,2.0,0.0,4.0,11.81,...,168.0,1.0,1.0,0.0,1.0,1.0,0.0,0.00,0.0,False
27774,32428,af51e40269e14424a5f08ed86a4d6437,offer received,504,9b98b8c7a33c4b65b9aebfe6a799e6d9,3.0,3.0,1.0,14.0,45.42,...,168.0,1.0,1.0,1.0,0.0,1.0,0.0,3.19,2.0,False
11131,13027,4713f6c6302741c58dcf928f40b1179f,offer received,504,2298d6c36e964ae4a3e7e9706d1fb8c2,1.0,1.0,1.0,4.0,7.45,...,168.0,1.0,1.0,1.0,1.0,0.0,1.0,3.64,2.0,False
11130,13025,471126123fb940acbed5cb8c0f395d0d,offer received,504,0b1e1539f2cc45b7b9fa7c272da2e1d7,2.0,2.0,1.0,1.0,33.62,...,240.0,1.0,0.0,1.0,0.0,0.0,1.0,0.00,0.0,False
27622,32246,ae4e5d8d13d74a2eba8ed2d049c1a0c1,offer received,504,ae264e3637204a6fb9bb56bc8210ddfd,0.0,0.0,0.0,1.0,1.09,...,168.0,1.0,1.0,0.0,1.0,1.0,0.0,21.44,10.0,False


In [35]:
test_data.completed.value_counts()

False    4175
True     3835
Name: completed, dtype: int64

In [36]:
train_data.shape

(32036, 36)

In [37]:
test_data.shape

(8010, 36)

In [38]:
X_train = train_data.drop('completed', axis=1) #train file, remove target value
y_train = train_data["completed"] #target value, binary True/False

X_test = test_data.drop('completed', axis=1) #test file, remove target value
y_test = test_data["completed"] #target value, binary True/False

In [39]:
y_test.head()

11133    False
27774    False
11131    False
11130    False
27622    False
Name: completed, dtype: bool

**Modelling** <br>
Built ML model, which will classify if user complete received offer or not.

In [40]:
def run_model(model, X_train, y_train, X_test, y_test, feats):
    """
    Function to run ML model.

    Input:
    model - model name
    X_train - train dataset
    y_train - train target value
    X_test - test dataset
    y_test - test target value
    feats - feats used in train/test dataset

    Output:
    precission - classification metrics, minimizing false positive
    """
    X_train = X_train[feats]
    X_test = X_test[feats]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    precision = precision_score(y_test, y_pred)
    return precision

In [41]:
def get_feats(df, black_list):
	"""
	Function which select numerous features to training dataset.
	Input:
	df -main dataframe
	black_list - list of features, which should be excluded from the final list

	Output:
	list of numerous features
	"""
	num_feats = df.select_dtypes(np.number).columns
	return [x for x in num_feats if x not in black_list]

In [42]:
#create list of numerous features used in modelling
black_list = ['completed', 'Unnamed: 0', 'time', 'last_amount','last_reward']
feats = get_feats(df, black_list)
feats

['cumsum_offer_received',
 'cumsum_offer_viewed',
 'cumsum_offer_completed',
 'cumsum_transaction_wo_offer',
 'cumsum_amount_wo_offer',
 'cumsum_amount_w_offer',
 'cumsum_reward',
 'gender_F',
 'gender_M',
 'gender_ND',
 'gender_O',
 'age_lower_than_26',
 'age_26-35',
 'age_36-45',
 'age_46-55',
 'age_56-65',
 'age_higher_than_65',
 'income',
 'member_since_months',
 'difficulty',
 'duration_days',
 'duration_hours',
 'channels_email',
 'channels_mobile',
 'channels_web',
 'channels_social',
 'offer_type_bogo',
 'offer_type_discount']

In [45]:
#perform modelling for 3 default models
models = [
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    xgb.XGBClassifier(verbosity=0),
]

for model in models:
    precision = run_model(model, X_train, y_train, X_test, y_test, feats)
    print(precision, model)

0.6370666001496632 DecisionTreeClassifier()
0.7243947858472998 RandomForestClassifier()
0.7379191321499013 XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


In [46]:
#print feature importance
X = X_train[feats]
model = RandomForestClassifier()
model.fit(X, y_train)

eli5.show_weights(model, feature_names=feats, top=50)

Weight,Feature
0.2189 ± 0.0166,member_since_months
0.2165 ± 0.0582,income
0.1132 ± 0.0390,cumsum_amount_wo_offer
0.0602 ± 0.0417,cumsum_amount_w_offer
0.0533 ± 0.0081,cumsum_transaction_wo_offer
0.0410 ± 0.0062,cumsum_offer_received
0.0365 ± 0.0578,gender_ND
0.0345 ± 0.0054,cumsum_offer_viewed
0.0282 ± 0.0249,cumsum_reward
0.0258 ± 0.0107,difficulty


**Grid Search** <br>
Improve hyperparameters for 2 the best models: Random Forest and XGBoost

In [47]:
def perform_gs(model):
  """
  Function which perform Grid Search for seleted model

  Input:
  model - model where hyperparameters should be improved

  Output:
  best hyperparameters choosed using grid search
  totabl score
  """
  param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
  }

  # Create the GridSearchCV object
  grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)

  # Fit the grid search to your data
  grid_search.fit(X, y_train)

  # Print the best hyperparameters and their corresponding score
  print("Best Hyperparameters: ", grid_search.best_params_)
  print("Best Score: ", grid_search.best_score_)
  return grid_search.best_params_, grid_search.best_score_

In [ ]:
#grid search for Random Forest
rf_best_params, rf_best_score = perform_gs(RandomForestClassifier())
print(rf_best_params, rf_best_score)
xgb_best_params,  = perform_gs(xgb.XGBClassifier(verbosity=0))
print(xgb_best_params, xgb_best_score)

In [ ]:
#test hyperparameters from Grid Search
precision = run_model(RandomForestClassifier(max_depth= 20, min_samples_leaf= 4, min_samples_split= 2, n_estimators= 200), X_train, y_train, X_test, y_test, feats)
precision

In [ ]:
#test hyperparameters from Grid Search
precision = run_model( xgb.XGBClassifier(max_depth= 5, min_samples_leaf= 1, min_samples_split= 2, n_estimators= 50), X_train, y_train, X_test, y_test, feats)
precision